In [1]:
import os
import sys

import time
import numpy as np
import pandas as pd
import yaml
import knpackage.toolbox as kn

from IPython.display import display
import ipywidgets as widgets

# sys.path.insert(1, '../src')
# import KnowEnG_graphics as gu
sys.path.insert(1, '../../Data_Cleanup_Pipeline/src')
import data_cleanup_toolbox as dc_tbx

sys.path.insert(1, '../../Samples_Clustering_Pipeline/src')
import sample_clustering_toolbox as sc_tbx

In [2]:
def git_clone_Samples_Clustering(pipelines_directory):
    """  clone samples clustering and data cleaning if they are not installed relative to the calling notebook """
    working_directory = os.getcwd()
    os.chdir(pipelines_directory)

    DC_name = 'Data_Cleanup_Pipeline'
    Data_Cleanup_Exists = False
    SC_name = 'Samples_Clustering_Pipeline'
    Samples_Clustering_Exists = False

    dir_listing = os.listdir()
    for d in dir_listing:
        if os.path.isdir(d):
            if d == DC_name:
                Data_Cleanup_Exists = True
            elif d == SC_name:
                Samples_Clustering_Exists = True

    if Data_Cleanup_Exists == False:
        dc_git_string = 'git clone https://github.com/KnowEnG/Data_Cleanup_Pipeline.git'
        os.system(dc_git_string)

    if Samples_Clustering_Exists == False:
        sc_git_string = 'git clone https://github.com/KnowEnG/Samples_Clustering_Pipeline.git'
        os.system(sc_git_string)

    os.chdir(working_directory)

In [3]:
def setup_run_dir(target_dir, REMOVE_RESULTS=False):
    """ setup directory for running a pipeline """
    if not os.path.isdir(run_dir):
        os.mkdir(run_dir)

    if os.path.isdir(results_dir) and REMOVE_RESULTS:
        os.system('rm ' + results_dir + '/*')
    elif not os.path.isdir(results_dir):
        os.mkdir(results_dir)

In [10]:
def data_cleanup(button):
    global data_cleanup_dict
    #     'phenotype_name_full_path':   '../../Samples_Clustering_Pipeline/data/spreadsheets/UCEC_phenotype.txt',
    data_cleanup_dict = {
        # 'spreadsheet_name_full_path': '../../Samples_Clustering_Pipeline/data/spreadsheets/tcga_ucec_somatic_mutation_data.df',
        # 'gg_network_name_full_path': '../../Samples_Clustering_Pipeline/data/networks/keg_ST90_4col.edge',
        'results_directory':          results_dir,
        'taxonid':                    '9606',
        'source_hint':                '',
        'pipeline_type':              'samples_clustering_pipeline',
        'redis_credential':
                                {'host': 'knowredis.knowhub.org',
                                'password': 'KnowEnG',
                                'port': '6380'}
    }

    for w in DC_widget_list:
        data_cleanup_dict[w.description[7:-1]] = os.path.join(target_dir,w.value)
    # print(data_cleanup_dict)
    
    SUCCESS, logging = dc_tbx.run_samples_clustering_pipeline(data_cleanup_dict)
    if SUCCESS == True:
        print('Data Cleanup completed successfully')
    else:
        print('Data Cleanup Failed - message log:\n')
        for k in logging:
            print(k)
            
    os.listdir(results_dir)

In [11]:
git_clone_Samples_Clustering(os.path.abspath('../../'))

target_dir = '../../Samples_Clustering_Pipeline/data/spreadsheets'

run_dir = os.path.join(target_dir, 'run_dir')
results_dir = os.path.join(run_dir, 'results')
setup_run_dir(target_dir)

os.system('cp ../../Samples_Clustering_Pipeline/data/networks/keg_ST90_4col.edge ' + target_dir)

#                                         Get list of (docker run -v) mounted files:
flist = os.listdir(target_dir)
FEXT = ['.tsv', '.txt', '.df','.edge']
my_file_list = []
for f in flist:
    if os.path.isfile(os.path.join(target_dir, f)):
        noNeed, f_ext = os.path.splitext(f)
        if f_ext in FEXT:
            my_file_list.append(f)

#                                         (docker run -v) mounted files was empty:
if len(my_file_list) <= 0:
    my_file_list.append('No Data')
    
DC_widget_list = []
DC_widget_list.append(widgets.Dropdown(
    options=my_file_list,
    value=my_file_list[0],
    description='Select spreadsheet_name_full_path:'
))

DC_widget_list.append(widgets.Dropdown(
    options=my_file_list,
    value=my_file_list[0],
    description='Select phenotype_name_full_path:'
))

DC_widget_list.append(widgets.Dropdown(
    options=my_file_list,
    value=my_file_list[0],
    description='Select gg_network_name_full_path:'
))

for w in DC_widget_list:
    display(w)
    
data_cleanup_button = widgets.Button(
    description='Data Cleanup',
    disabled=False,
    button_style='',
    tooltip='data cleanup button',
    )
data_cleanup_button.on_click(data_cleanup)
display(data_cleanup_button)

Data Cleanup Failed - check log messages
INFO: Successfully loaded input data: ../../Samples_Clustering_Pipeline/data/spreadsheets/tcga_ucec_somatic_mutation_data.df with 17547 row(s) and 248 column(s)
INFO: Start processing phenotype data.
INFO: Successfully loaded input data: ../../Samples_Clustering_Pipeline/data/spreadsheets/UCEC_phenotype.txt with 451 row(s) and 19 column(s)
INFO: Start to run sanity check for phenotype data.
INFO: No duplicate column name detected in this data set.
INFO: No duplicate row name detected in this data set.
INFO: Found 248 intersected gene(s) between phenotype and spreadsheet data.
INFO: Finished running sanity check for phenotype data.
INFO: Start processing user spreadsheet data.
INFO: Start to run sanity checks for user spreadsheet data.
INFO: No duplicate column name detected in this data set.
INFO: No duplicate row name detected in this data set.
INFO: Mapped 17490 gene(s) to ensemble name.
INFO: Unable to map 57 gene(s) to ensemble name.
INFO: F

In [6]:
try_dict = {'a': 23, 'b':'whodakine', 'c': 44}
for k, v in try_dict.items():
    print(k, ':', v)

a : 23
b : whodakine
c : 44


In [6]:
# samples_cluster_dict = {
#             'method': 'cc_net_nmf',
#             'spreadsheet_name_full_path': '../test/run_dir/results/tcga_ucec_somatic_mutation_data_ETL.tsv',
#             'phenotype_name_full_path':   '../test/run_dir/results/UCEC_phenotype_ETL.tsv',
#             'threshold': '10',
#             'gg_network_name_full_path':  '../data/networks/keg_ST90_4col.edge',
#             'results_directory':          '../../user_data/run_dir/results',
#             'tmp_directory':               '../../user_data/run_dir',
#             'rwr_max_iterations':         '100',
#             'rwr_convergence_tolerence':  '1.0e-4',
#             'rwr_restart_probability':    '0.7',
#             'rows_sampling_fraction':     '0.8',
#             'cols_sampling_fraction':     '0.8',
#             'number_of_bootstraps':       '4',
#             'number_of_clusters':         '3',
#             'nmf_conv_check_freq':        '50',
#             'nmf_max_invariance':         '200',
#             'nmf_max_iterations':         '10000',
#             'nmf_penalty_parameter':      '1400',
#             'top_number_of_genes':        '100',
#             'processing_method':          'parallel',
#             'parallelism':                '4'
#         }

method_list = ['nmf', 'cc_nmf', 'net_nmf', 'cc_net_nmf']
threshold_range = {'low':2, 'high':100, 'tip':'categorical vs numerical cutoff threshold'}
rwr_max_iterations_range = {'low':2, 'high':1000, 'tip':'random walk no convergence iteration limit'}
rwr_convergence_tolerence_range = {'low':1.0e-16, 'high':1000, 'tip':'minimum norm difference'}
rwr_restart_probability_range = {'low':0, 'high':1, 'tip': 'Vn+1 = alpha * N * Vn + (1-alpha) * Vo'}
rows_sampling_fraction_range = {'low':0, 'high':1, 'tip': 'bootstrap sampling fraction of spreadsheet rows'}
cols_sampling_fraction_range = {'low':0, 'high':1, 'tip': 'bootstrap sampling fraction of spreadsheet columns'}
number_of_bootstraps_range = {'low':1, 'high':2000, 'tip': 'more bootstrap samples == more run time'}
number_of_clusters_range = {'low':2, 'high':12, 'tip': 'more clusters == more run time'}

# optional parameters
nmf_conv_check_freq_range = {'low':1, 'high':1000, 'tip': 'more frequent checks == more run time'}
# nmf_max_invariance_range = 

# available clusters: AWS, CS Cluster
# available methods: serial, parallel, distribute

In [7]:
def samples_clustering(button):
    spreadsheet_name_post_clean = os.path.splitext(DC_widget_list[0].value)[0] + '_ETL.tsv'
    phenotype_name_post_clean = os.path.splitext(DC_widget_list[1].value)[0] + '_ETL.tsv'
    spreadsheet_name_post_clean = os.path.join(results_dir, spreadsheet_name_post_clean)
    phenotype_name_post_clean = os.path.join(results_dir, phenotype_name_post_clean)
    try:
        samples_cluster_dict = {
                 'spreadsheet_name_full_path': spreadsheet_name_post_clean,
                 'phenotype_name_full_path':   phenotype_name_post_clean,
                 'gg_network_name_full_path':  data_cleanup_dict['gg_network_name_full_path'],
                 'results_directory':          results_dir,
                 'run_directory':              run_dir,
        }

        for w in SC_widget_list:
            if not isinstance(w,widgets.Label):
                samples_cluster_dict[w.description[7:-1]] = w.value

        if samples_cluster_dict['method'] == 'cc_net_nmf':
            sc_tbx.run_cc_net_nmf(samples_cluster_dict)
    except NameError:
        print('You should run Data Cleanup Pipeline first! ')

In [8]:
SC_widget_list = []
SC_widget_list.append(widgets.Dropdown(
    options=method_list,
    value='cc_net_nmf',
    description='Select method:'
))

SC_widget_list.append(widgets.IntSlider(
    min=2, 
    max=100,  
    value=10, 
    description='Select threshold:'
))

SC_widget_list.append(widgets.IntSlider(
    min=2, 
    max=1000, 
    value=100, 
    description='Select rwr_max_iterations:'
))

SC_widget_list.append(widgets.BoundedFloatText(
    min=1.0e-16, 
    max=1000,  
    value=1e-4, 
    description='Select rwr_convergence_tolerence:'
))

SC_widget_list.append(widgets.FloatSlider(
    min=0, 
    max=1,  
    value=0.7, 
    description='Select rwr_restart_probability:'
))

SC_widget_list.append(widgets.FloatSlider(
    min=0, 
    max=1,  
    value=0.8, 
    description='Select rows_sampling_fraction:'
))

SC_widget_list.append(widgets.FloatSlider(
    min=0, 
    max=1, 
    value=0.8, 
    description='Select cols_sampling_fraction:'
))

SC_widget_list.append(widgets.IntSlider(
    min=1, 
    max=2000, 
    value=4, 
    description='Select number_of_bootstraps:'
))

SC_widget_list.append(widgets.IntSlider(
    min=2, 
    max=12,  
    value=3, 
    description='Select number_of_clusters:'
))

SC_widget_list.append(widgets.Label(
    value='Optional Parameters: '
))

SC_widget_list.append(widgets.IntSlider(
    min=1, 
    max=1000,
    value=50, 
    description='Select nmf_conv_check_freq:'
))

SC_widget_list.append(widgets.Dropdown(
    options=['serial', 'parallel', 'distribute'],
    value='parallel',
    description='Select processing_method:'
))

SC_widget_list.append(widgets.IntSlider(
    min=1, 
    max=4,  
    value=4, 
    description='Select parallelism:'
))

for w in SC_widget_list:
    display(w)
        
samples_clustering_button = widgets.Button(
    description='Samples Clustering',
    disabled=False,
    button_style='',
    tooltip='samples clustering button',
    )
samples_clustering_button.on_click(samples_clustering)
display(samples_clustering_button)

Unexpected error during reading input file ../../Samples_Clustering_Pipeline/data/spreadsheets/run_dir/results/tcga_ucec_somatic_mutation_data_ETL.tsv: <class 'FileNotFoundError'>


FileNotFoundError: File b'../../Samples_Clustering_Pipeline/data/spreadsheets/run_dir/results/tcga_ucec_somatic_mutation_data_ETL.tsv' does not exist